In [1]:
import pandas as pd
import numpy as np
from astropy.time import Time
from scipy import interpolate
from scipy.stats import spearmanr
import matplotlib.pyplot as plot
from geopy.distance import distance as gdistance
from sklearn.preprocessing import StandardScaler as SCALER
import itertools
import seaborn as sns
import immigration_data
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_validate,GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import xgboost as xgb

In [42]:
import immigration_data
import importlib
importlib.reload(immigration_data)
imm = immigration_data.immigration_data()
X,y,d = imm.get_training_data()
year_pred = imm.year_pred


Can not use standard cross validation because spatial correlations lead to data leakage. Have to separate train and test sets separating by years. The routine below does that.

In [3]:
def CV_immigration_data(reg, X, y, year_pred):

    ind_train1 = (year_pred != 2005) & (year_pred != 2006)
    ind_test1  = (year_pred == 2005) | (year_pred == 2006)    
    ind_train2 = (year_pred != 2007) & (year_pred != 2008)
    ind_test2  = (year_pred == 2007) | (year_pred == 2008)
    ind_train3 = (year_pred != 2009) & (year_pred != 2010)
    ind_test3  = (year_pred == 2009) | (year_pred == 2010)
    ind_train4 = (year_pred != 2011) & (year_pred != 2012)
    ind_test4  = (year_pred == 2011) | (year_pred == 2012)
    ind_train5 = (year_pred != 2013) & (year_pred != 2014)
    ind_test5  = (year_pred == 2013) | (year_pred == 2014)
    ind_train6 = (year_pred != 2015) & (year_pred != 2016) & (year_pred != 2017)
    ind_test6  = (year_pred == 2015) | (year_pred == 2016) | (year_pred == 2017)
    
    Xtrain1 = X[ind_train1] 
    Xtrain2 = X[ind_train2]
    Xtrain3 = X[ind_train3]
    Xtrain4 = X[ind_train4]
    Xtrain5 = X[ind_train5]
    Xtrain6 = X[ind_train6]

    Xtest1 = X[ind_test1]
    Xtest2 = X[ind_test2]
    Xtest3 = X[ind_test3]
    Xtest4 = X[ind_test4]
    Xtest5 = X[ind_test5]
    Xtest6 = X[ind_test6]
    
    ytrain1 = y[ind_train1]
    ytrain2 = y[ind_train2]
    ytrain3 = y[ind_train3]
    ytrain4 = y[ind_train4]
    ytrain5 = y[ind_train5]
    ytrain6 = y[ind_train6]

    ytest1 = y[ind_test1]
    ytest2 = y[ind_test2]
    ytest3 = y[ind_test3]
    ytest4 = y[ind_test4]
    ytest5 = y[ind_test5]
    ytest6 = y[ind_test6]
    
    reg.fit(Xtrain1, ytrain1)
    ypred1 = reg.predict(Xtest1)
    #score1 = mean_absolute_error(ytest1, ypred1)
    score1 = r2_score(ytest1, ypred1)

    reg.fit(Xtrain2, ytrain2)
    ypred2 = reg.predict(Xtest2)
    score2 = r2_score(ytest2, ypred2)

    reg.fit(Xtrain3, ytrain3)
    ypred3 = reg.predict(Xtest3)
    score3 = r2_score(ytest3, ypred3)
  
    reg.fit(Xtrain4, ytrain4)
    ypred4 = reg.predict(Xtest4)
    score4 = r2_score(ytest4, ypred4)

    reg.fit(Xtrain5, ytrain5)
    ypred5 = reg.predict(Xtest5)
    score5 = r2_score(ytest5, ypred5)

    reg.fit(Xtrain5, ytrain5)
    ypred5 = reg.predict(Xtest5)
    score5 = r2_score(ytest5, ypred5)

    reg.fit(Xtrain6, ytrain6)
    ypred6 = reg.predict(Xtest6)
    score6 = r2_score(ytest6, ypred6)
    
    score_out = np.median([score1, score2, score3, score4, score5, score6])
    
    return score_out


The optimization procedure is iterative. The approach is to fit all the features, do hyper parameter optimization, remove the least important feature, calcualte score, redo hyper parameter optimization, remove least important feature etc etc. I only do 20 evaluations for each iteration due to computing limitations. Ideally, the number of evaluations per iteration would be larger.

In [12]:
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK, space_eval


def get_best_indices(reg_trained, X, y, year_pred):

    thresholds = np.sort(reg_trained.feature_importances_)
    feature_importance = reg_trained.feature_importances_

    master_index = np.arange(len(X[0,:]))

    score_out = []
    index_out = []
    
    nindex = len(master_index)
    X_in = X.copy()
    
    while nindex > 2:
        index = np.where((feature_importance > thresholds[0]) & (feature_importance > 0))[0]
        el = len(index)
        
        X_in = X_in[:,index]
        master_index = master_index[index]
        index_out.append(master_index)
        # Run hyperopt optimization

        def objective(para):
            # print(para['max_depth'],para['learning_rate'])
            reg = xgb.XGBRegressor(**para,objective='reg:squarederror')

            testScore = CV_immigration_data(reg, X_in, y, year_pred)
            #print(testScore)
            return {'loss': -1 * testScore, 'status': STATUS_OK}

        
        trials = Trials()
        space = {
            'learning_rate':    hp.choice('learning_rate',    np.arange(0.1, 0.2, 0.02)),
            'max_depth':        hp.choice('max_depth',        np.arange(6, 10, 1, dtype=int)),
            'min_child_weight': hp.choice('min_child_weight', np.arange(4, 8, 1, dtype=int)),
            'colsample_bytree': hp.choice('colsample_bytree', np.arange(0.3, 0.8, 0.1)),
            'subsample':        hp.uniform('subsample', 0.75, 0.95),
            'n_estimators':     hp.choice('n_estimators',      np.arange(20,100,10))   }
        result = fmin(fn = objective, space = space, algo = tpe.suggest,
                      trials=trials, max_evals=20)
        
        reg_trained = get_best_xgb_model.get_best_xgb_model(space_eval(space, result))
        reg_trained.fit(X_in, y)
        feature_importance = reg_trained.feature_importances_
        thresholds = np.sort(reg_trained.feature_importances_)

        
        score = CV_immigration_data(reg_trained, X_in , y, year_pred)
        print('N Features = ' ,el , 'R2 score = ', score)
        score_out.append(score)
        
        nindex = len(index)

    return score_out, index_out

Determine the best score and the corresponding indices. For this particular run, the best set of indices was at iteration 80. Rerunning this with larger number of evaluations may produce different results and should be tested. After finding the best set of indices, run a higher number of evaluations to find the best set of hyper parameters.

In [ ]:
index_sel = index_out[80]

In [33]:
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK, space_eval

def objective(para):
    # print(para['max_depth'],para['learning_rate'])
    reg = xgb.XGBRegressor(**para,objective='reg:squarederror')

    testScore = CV_immigration_data(reg, X[:, index_sel], y, year_pred)
    #print(testScore)
    return {'loss': -1 * testScore, 'status': STATUS_OK}
                                

# Run hyperopt optimization
trials = Trials()
space = {
    'learning_rate':    hp.choice('learning_rate',    np.arange(0.06, 0.2, 0.02)),
    'max_depth':        hp.choice('max_depth',        np.arange(4, 14, 1, dtype=int)),
    'min_child_weight': hp.choice('min_child_weight', np.arange(2, 12, 1, dtype=int)),
    'colsample_bytree': hp.choice('colsample_bytree', np.arange(0.3, 0.8, 0.1)),
    'subsample':        hp.uniform('subsample', 0.75, 0.99),
    'n_estimators':     hp.choice('n_estimators',      np.arange(20,100,5))   }
result = fmin(fn = objective, space = space, algo = tpe.suggest,
              trials=trials, max_evals=300)





100%|██████████| 300/300 [3:50:01<00:00, 37.33s/it, best loss: -0.1923667965474231]   


para is the best set of hyper parameters. I took these from this hyper parameter/feature selection optimization sequence and hard coded them into the get_best_xgb_model.py routine. If rerunning the hyper parameter optimization changes the result, modify that routine.

In [34]:
para = space_eval(space, result)
print(para)

{'colsample_bytree': 0.4,
 'learning_rate': 0.1,
 'max_depth': 6,
 'min_child_weight': 8,
 'n_estimators': 65,
 'subsample': 0.9174674408083486}